In [3]:
print('Success vs Failure')

Notebook on


In [4]:
import glob
import pandas as pd
import figs
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import chisquare
from scipy import stats
import nltk

In [5]:
import json
import figs
import glob

In [6]:
from sklearn import svm, linear_model, cluster
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neural_network import MLPClassifier,BernoulliRBM
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, balanced_accuracy_score
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn import naive_bayes, ensemble
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

In [7]:
def path2texts(path):
    
    """ Path to success or failure """
    
    texts = []
    for el in glob.glob(path):
        try:
            text = open(el).read()
            title = el.split('_')[-1]
            texts.append((title,text))
        except:
            ''
            
    return texts



def computing_features(texts):
    
    """Function computing the features:
        - Sentimental arcs
        - Sentimental arcs' average Hurst exponent
        - Type-Token Ratio
    """
    
    h_values = []
    ttr_values = []
    arcs = []
    
    for double in texts: 
        
        title,text = double
    
        print(title, len(text))

        words = nltk.wordpunct_tokenize(text)
        ttr = len(list(set(words)))/len(words)
        
        arc = figs.sentimarc_sid(text)
        #if len(arc)>300: 
        h = figs.get_Hurst(arc)

        arcs.append(arc)
        h_values.append(h)
        ttr_values.append(ttr)
        
    return h_values, ttr_values, arcs


def features_and_labels(h_values, ttr_values, arcs, label_name):
    labels = []
    X = []
    
    """Function aggregating the features"""
    
    for i in range(len(h_values)):
        labels.append(label_name)
        l=len(arcs[i])
        n=int(l/6)
        X.append((h_values[i], abs(h_values[i]-0.6),
                  ttr_values[i], arcs[i][-1],
                  np.mean(arcs[i]),
                   np.mean(arcs[i][-n:]), np.mean(arcs[i][:n])
                 ))
        
    return X,labels


def path2features(path, label_name):
    
    """Feed an emotion flow path and a label name"""
    
    texts_set = path2texts(path)
    
    hs,ts,ar = computing_features(texts_set)
    
    X, labels = features_and_labels(hs,ts,ar,label_name)
    
    return X, labels

# Making or uploading book features

path_to_success = 'emotion_flow-master/data/data_all/Fiction/success/*'
path_to_failure = 'emotion_flow-master/data/data_all/Fiction/failure/*'


X_success, y_success = path2features(path_to_success, 's')

print()


X_failure, y_faillure = path2features(path_to_failure, 'f')

done


with open('eacl_fiction_success.json', 'w') as fp:
    json.dump(X_success, fp)
    
with open('eacl_fiction_failure.json', 'w') as fp:
    json.dump(X_failure, fp)

In [83]:
# Upload precomputed scores

import json
 
# Opening JSON file
with open('fiction_success.json') as json_file:
    X_success = json.load(json_file)
    
with open('fiction_failure.json') as json_file:
    X_failure = json.load(json_file)
    
y_success, y_failure = ['s' for el in X_success], ['f' for el in X_failure]

In [84]:
len(y_success), len(y_failure)

(81, 30)

In [85]:
X = X_success + X_failure
labels = y_success + y_failure

In [86]:
len(X) ,len(labels)

(111, 111)

# Classifications

In [88]:
from sklearn import svm, linear_model, cluster
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neural_network import MLPClassifier,BernoulliRBM
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, balanced_accuracy_score
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn import naive_bayes, ensemble
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_val_score, KFold, ShuffleSplit, RepeatedKFold

In [230]:
classifiers = [linear_model.LogisticRegression,
               GaussianNB,
               LinearDiscriminantAnalysis,
               RandomForestClassifier, 
               svm.SVC,GaussianProcessClassifier,
               QuadraticDiscriminantAnalysis,
                ]
ensemble_classifiers = [ensemble.StackingClassifier, ensemble.VotingClassifier]

In [232]:
classifiers_scores = {}

In [233]:
for classifier in classifiers:
    
    clf = classifier()
    clf_name = clf.__str__()
    
    print(clf_name)
    
    cv_ = RepeatedKFold(n_splits=10, n_repeats=2, random_state=10)
    
    average_repeated_f_score = np.mean(cross_val_score(clf, X, labels, cv=cv_, scoring='f1_weighted', n_jobs=-1))
    average_tenfold_f_score = np.mean(cross_val_score(clf, X, labels, cv=5,scoring='f1_weighted'))
    
    X_train , X_test , y_train, y_test = train_test_split(X, labels, random_state=0, test_size=0.2)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
    single_f_score = f1_score(y_test,y_pred,average='weighted') 
    binary_f_score = f1_score(y_pred,y_test,pos_label='s',average='binary')
    
    print(single_f_score)
    
    classifiers_scores[clf_name] = [average_repeated_f_score, average_tenfold_f_score,
                                    balanced_accuracy, single_f_score, binary_f_score]

LogisticRegression()
0.6282608695652173
GaussianNB()
0.7202898550724638
LinearDiscriminantAnalysis()
0.755189972581277
RandomForestClassifier()
0.8135265700483091
SVC()
0.6282608695652173
GaussianProcessClassifier()
0.6282608695652173
QuadraticDiscriminantAnalysis()
0.8135265700483091


In [234]:
for classifier in ensemble_classifiers:
    
    clf = classifier(estimators = [('q',QuadraticDiscriminantAnalysis()), ('gaussian',GaussianNB()),('linear2', LinearDiscriminantAnalysis()), ('forest',RandomForestClassifier())])
    clf_name = clf.__str__()
    
    print(clf_name)
    
    cv_ = RepeatedKFold(n_splits=10, n_repeats=2, random_state=10)
    
    average_repeated_f_score = np.mean(cross_val_score(clf, X, labels, cv=cv_, scoring='f1_weighted', n_jobs=-1))
    average_tenfold_f_score = np.mean(cross_val_score(clf, X, labels, cv=5,scoring='f1_weighted'))
    
    X_train , X_test , y_train, y_test = train_test_split(X, labels, random_state=0, test_size=0.2)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
    single_f_score = f1_score(y_test,y_pred,average='weighted') 
    binary_f_score = f1_score(y_pred,y_test,pos_label='s',average='binary')
    
    print(single_f_score)
    
    classifiers_scores[clf_name] = [average_repeated_f_score, average_tenfold_f_score,
                                    balanced_accuracy, single_f_score, binary_f_score]

StackingClassifier(estimators=[('q', QuadraticDiscriminantAnalysis()),
                               ('gaussian', GaussianNB()),
                               ('linear2', LinearDiscriminantAnalysis()),
                               ('forest', RandomForestClassifier())])
0.755189972581277
VotingClassifier(estimators=[('q', QuadraticDiscriminantAnalysis()),
                             ('gaussian', GaussianNB()),
                             ('linear2', LinearDiscriminantAnalysis()),
                             ('forest', RandomForestClassifier())])
0.8135265700483091


In [236]:
classifiers_scores

{'LogisticRegression()': [0.6199245886552387,
  0.6156043270230913,
  0.5,
  0.6282608695652173,
  0.85],
 'GaussianNB()': [0.7016261129651223,
  0.7202568406916233,
  0.6078431372549019,
  0.7202898550724638,
  0.8333333333333333],
 'LinearDiscriminantAnalysis()': [0.7178715505340892,
  0.7266352970700797,
  0.6372549019607843,
  0.755189972581277,
  0.8648648648648648],
 'RandomForestClassifier()': [0.7040141463283879,
  0.6983602286547589,
  0.7205882352941176,
  0.8135265700483091,
  0.8888888888888888],
 'SVC()': [0.6199245886552387,
  0.6156043270230913,
  0.5,
  0.6282608695652173,
  0.85],
 'GaussianProcessClassifier()': [0.6199245886552387,
  0.6156043270230913,
  0.5,
  0.6282608695652173,
  0.85],
 'QuadraticDiscriminantAnalysis()': [0.7063788035150265,
  0.7027705934487554,
  0.7205882352941176,
  0.8135265700483091,
  0.8888888888888888],
 "StackingClassifier(estimators=[('q', QuadraticDiscriminantAnalysis()),\n                               ('gaussian', GaussianNB()),\n  

# One-classifier scores

In [237]:
clf = QuadraticDiscriminantAnalysis()

In [238]:
# Ten-fold CV

np.mean(cross_val_score(clf, X, labels, cv=10, scoring='f1_micro'))

0.7386363636363636

In [239]:
# Ten fold CV with repeats
cv_ = RepeatedKFold(n_splits=10, n_repeats=20, random_state=10)
np.mean(cross_val_score(clf, X, labels, cv=cv_, scoring='f1_micro', n_jobs=-1))

0.7506439393939394

In [240]:
# average over several cross validations
scores = []
for i in range(4):
    ss = cross_val_score(clf, X, labels, cv=6,scoring='f1_micro')
    scores+=list(ss)
np.mean(scores)

0.7465886939571149

In [241]:
# One time classification

X_train , X_test , y_train, y_test = train_test_split(X, labels, random_state=6, test_size=0.1)

print(len(X_train), len(X_test), len(X))

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("f score:\t", f1_score(y_test,y_pred,average='micro')) #balanced_accuracy_score(y_pred,y_test)
print("balanced acc.:\t",balanced_accuracy_score(y_test, y_pred))

99 12 111
f score:	 0.9166666666666666
balanced acc.:	 0.95


In [242]:
len(y_pred)

12

In [243]:
print("binary f score:\t",f1_score(y_pred,y_test,pos_label='s',average='binary'))

binary f score:	 0.9473684210526316


In [244]:
for i in range(len(y_pred)): print(y_pred[i],y_test[i])

s s
s s
s s
f f
s s
s s
f f
s s
s s
s s
s s
f s
